In [3]:
import pandas as pd
import numpy as np

import geopandas as gpd

from shapely.geometry import Point
import rtree
import pickle

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import secret_key

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [85]:
mta_df = pd.read_csv("data/mta_turnstile/turnstile_170107.txt")
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170114.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170121.txt"))
mta_df = mta_df.append(pd.read_csv("data/mta_turnstile/turnstile_170128.txt"))

In [86]:
mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,03:00:00,REGULAR,5991546,2028378
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,07:00:00,REGULAR,5991565,2028389
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,11:00:00,REGULAR,5991644,2028441
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,15:00:00,REGULAR,5991971,2028502
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/31/2016,19:00:00,REGULAR,5992418,2028543


In [89]:
mta_df['DATE'] = pd.to_datetime(mta_df.DATE)

In [90]:
mta_df['month'] = mta_df.DATE.dt.month

In [91]:
mta_df['year'] = mta_df.DATE.dt.year

In [92]:
mta_df['day_of_week'] = mta_df.DATE.dt.weekday_name

In [93]:
mta_df['TIME'] = pd.to_datetime(mta_df['TIME'], format='%H:%M:%S')

In [94]:
mta_df['time_of_day'] = mta_df.TIME.dt.hour

In [14]:
mta_df['DATE'] = pd.to_datetime(mta_df.DATE).dt.date

In [23]:
import requests
import json
response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address=103+st+station+new+york+line+1&key='+secret_key.google_api_key)

In [24]:
#response.json()[['results']
test = json.loads(response.text)

In [27]:
test['results'][0]['geometry']['location']['lat'], test['results'][0]['geometry']['location']['lng']

(40.7902308, -73.9477564)

In [28]:
mta_loc = mta_df.groupby(['STATION', 'LINENAME', 'DIVISION'])['UNIT'].count().reset_index()

In [31]:
mta_loc = mta_loc.drop('UNIT', axis=1)

In [35]:
mta_loc.head()

,STATION,LINENAME,DIVISION
0,1 AV,L,BMT
1,103 ST,1,IRT
2,103 ST,6,IRT
3,103 ST,BC,IND
4,103 ST-CORONA,7,IRT


In [53]:
import time

In [69]:
for index, loc in mta_loc.iterrows():
    try:
        station = mta_loc.STATION[index]
        linename = mta_loc.LINENAME[index]
        division = mta_loc.DIVISION[index]

        search_query = "https://maps.googleapis.com/maps/api/geocode/json?address="
        for word in station.split(" "):
            search_query += word + '+'

        search_query += 'station+' + linename + '+new+york+'
        search_query += division + '&key=' + secret_key.google_api_key
#         print(search_query)
        response = requests.get(search_query)
        result = json.loads(response.text)

        mta_loc.loc[[index], 'lat'] = result['results'][0]['geometry']['location']['lat']
        mta_loc.loc[[index], 'long'] = result['results'][0]['geometry']['location']['lng']
    except:
        print(mta_loc.STATION[index])
    time.sleep(1)
        
    

111 ST
CENTRAL AV
HEWES ST


In [103]:
mta_loc[mta_loc.LINENAME == 'NQR456W']

,STATION,LINENAME,DIVISION,lat,long
124,59 ST,NQR456W,BMT,40.641568,-74.017678


In [97]:
mta_loc['lat'][328] = 40.706693

/home/iniyavan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [95]:
mta_loc['long'][328] = -73.952892

/home/iniyavan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
mta_loc['long'][244]

-73.927481

In [102]:
mta_loc.head()

,STATION,LINENAME,DIVISION,lat,long
0,1 AV,L,BMT,40.763368,-73.959240
1,103 ST,1,IRT,40.773697,-73.982229
2,103 ST,6,IRT,40.790231,-73.947756
3,103 ST,BC,IND,40.796289,-73.961658
4,103 ST-CORONA,7,IRT,40.749780,-73.862646


In [153]:
f = open('storage/mta_loc.pckl', 'wb')
pickle.dump(mta_loc, f)
f.close()

In [53]:
f = open('storage/mta_loc.pckl', 'rb')
mta_loc = pickle.load(f)
f.close()

In [5]:
mta_loc.shape

(478, 5)

In [98]:
test = pd.merge(mta_df, mta_loc, on=['STATION', 'LINENAME', 'DIVISION'])

In [107]:
test.shape

(777103, 16)

In [99]:
test.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week,time_of_day,lat,long
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 03:00:00,REGULAR,5991546,2028378,12,2016,Saturday,3,40.641568,-74.017678
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 07:00:00,REGULAR,5991565,2028389,12,2016,Saturday,7,40.641568,-74.017678
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 11:00:00,REGULAR,5991644,2028441,12,2016,Saturday,11,40.641568,-74.017678
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 15:00:00,REGULAR,5991971,2028502,12,2016,Saturday,15,40.641568,-74.017678
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 19:00:00,REGULAR,5992418,2028543,12,2016,Saturday,19,40.641568,-74.017678


In [100]:
test['zone'] = test['STATION'] + " " + test['LINENAME'] + " " + test['DIVISION'] 

In [101]:
test.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week,time_of_day,lat,long,zone
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 03:00:00,REGULAR,5991546,2028378,12,2016,Saturday,3,40.641568,-74.017678,59 ST NQR456W BMT
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 07:00:00,REGULAR,5991565,2028389,12,2016,Saturday,7,40.641568,-74.017678,59 ST NQR456W BMT
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 11:00:00,REGULAR,5991644,2028441,12,2016,Saturday,11,40.641568,-74.017678,59 ST NQR456W BMT
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 15:00:00,REGULAR,5991971,2028502,12,2016,Saturday,15,40.641568,-74.017678,59 ST NQR456W BMT
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 19:00:00,REGULAR,5992418,2028543,12,2016,Saturday,19,40.641568,-74.017678,59 ST NQR456W BMT


In [2]:
bike_df = pd.read_csv("data/201701-citibike-tripdata.csv.zip")

In [3]:
bike_df['Start Time'] = pd.to_datetime(bike_df['Start Time'])

In [4]:
bike_df['Stop Time'] = pd.to_datetime(bike_df['Stop Time'])

In [5]:
bike_df['month'] = bike_df['Start Time'].dt.month

In [6]:
bike_df['year'] = bike_df['Start Time'].dt.year

In [7]:
bike_df['day_of_week'] = bike_df['Start Time'].dt.weekday_name

In [8]:
bike_df['time_of_day'] = bike_df['Start Time'].dt.hour

In [45]:
bike_df['DATE'] = bike_df['Start Time'].dt.date

In [57]:
def bin_time(x):
    if x.hour <= 3 or x.hour == 24:
        return 3
    elif x.hour < 7:
        return 7
    elif x.hour < 11:
        return 11
    elif x.hour < 15:
        return 15
    elif x.hour < 19:
        return 19
    else:
        return 23

bike_df["time_of_day"] = bike_df["Start Time"].apply(bin_time)

In [60]:
bike_df.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,month,year,day_of_week,time_of_day,zone,DATE
0,680,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2,1,2017,Sunday,3,81 ST-MUSEUM BC IND,2017-01-01
1,1282,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2,1,2017,Sunday,3,8 ST-NYU NRW BMT,2017-01-01
6,559,2017-01-01 00:05:00,2017-01-01 00:14:20,499,Broadway & W 60 St,40.769155,-73.981918,479,9 Ave & W 45 St,40.760193,-73.991255,27294,Subscriber,1973.0,1,1,2017,Sunday,3,59 ST COLUMBUS 1ABCD IRT,2017-01-01
7,826,2017-01-01 00:05:37,2017-01-01 00:19:24,362,Broadway & W 37 St,40.751726,-73.987535,445,E 10 St & Avenue A,40.727408,-73.981420,23288,Subscriber,1977.0,2,1,2017,Sunday,3,34 ST-PENN STA 123 IRT,2017-01-01
8,255,2017-01-01 00:05:47,2017-01-01 00:10:02,430,York St & Jay St,40.701485,-73.986569,242,Carlton Ave & Flushing Ave,40.697787,-73.973736,25041,Subscriber,1989.0,1,1,2017,Sunday,3,YORK ST F IND,2017-01-01


In [11]:
mta_loc['zone'] = mta_loc['STATION'] + " " + mta_loc['LINENAME'] + " " + mta_loc['DIVISION'] 

In [49]:
mta_loc.head()

,STATION,LINENAME,DIVISION,lat,long,zone
0,1 AV,L,BMT,40.763368,-73.959240,1 AV L BMT
1,103 ST,1,IRT,40.773697,-73.982229,103 ST 1 IRT
2,103 ST,6,IRT,40.790231,-73.947756,103 ST 6 IRT
3,103 ST,BC,IND,40.796289,-73.961658,103 ST BC IND
4,103 ST-CORONA,7,IRT,40.749780,-73.862646,103 ST-CORONA 7 IRT


In [95]:
mini_bike = bike_df.head(10)
copy_mini = mini_bike.copy()


In [21]:
def biketomtazone(lat, long):
    
#     station_lat = 40.781979
#     station_long = -73.972014
    for index, location in mta_loc.iterrows():
        station_lat = location['lat']
        station_long = location['long']
        if( ((station_lat + 0.002000) >= lat)  and ((station_lat - 0.002000) <= lat) ):
            if( ((station_long + 0.003000) >= long) and ((station_long - 0.003000) <= long) ):
                return location['zone']
    return False
#     #left-top-corner
#     print(lat + 0.002000, ",",long - 0.003000)
    
#     #right-top-corner
#     print(lat + 0.002000, ",",long + 0.003000)
    
#     #right-bottom-corner
#     print(lat - 0.002000,",", long + 0.003000)
    
#     #left-bottom-corner
#     print(lat - 0.002000, ",",long - 0.003000)
    
# biketomtazone(40.782750, -73.971370)
# biketomtazone(40.776829, -73.963888)

In [12]:
from threading import Thread, Lock

mutex = Lock()

def biketomtazone(temp):
    index = temp[0]
    row = temp[1]
    
    if( index%10000 == 0):
        print(index)
        
    lat = row['Start Station Latitude']
    long = row['Start Station Longitude']
#     station_lat = 40.781979
#     station_long = -73.972014
    for index, location in mta_loc.iterrows():
        station_lat = location['lat']
        station_long = location['long']
        if( ((station_lat + 0.002000) >= lat)  and ((station_lat - 0.002000) <= lat) ):
            if( ((station_long + 0.003000) >= long) and ((station_long - 0.003000) <= long) ):
#                 print(location['zone'])
                return location['zone']
#                 mutex.acquire()
#                 copy_mini.loc[index, 'zone'] = location['zone']
#                 mutex.release()
#                 print(copy_mini)
#                 break
    return False


In [22]:
for index, ride in mini_bike.iterrows():
    result = biketomtazone(ride['Start Station Latitude'],ride['Start Station Longitude'])
    if(result is not False):
        mini_bike.loc[index, 'zone'] = result
    if(index % 1000 == 0):
        print(index)
       

/home/iniyavan/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0


In [ ]:
from multiprocessing import Pool

p = Pool()
result = p.map(biketomtazone, bike_df.iterrows())
p.close()
p.join()

# print(result)
result = np.array(result)
bike_df['zone'] = result.T
#     if(result is not False):
#         mini_bike.loc[index, 'zone'] = result
#     if(index % 10 == 0):
#         print(index)
       

In [ ]:
data = merged_df.merge(bike_df, on=['DOW', 'TOD', 'taxi_zone'], how='right')

In [21]:
f = open('storage/bike_df_loc.pckl', 'wb')
pickle.dump(bike_df, f)
f.close()

In [35]:
f = open('storage/bike_df_loc.pckl', 'rb')
bike_df = pickle.load(f)
f.close()

In [6]:
mta_df.head()

,zone,DATE,time_of_day,DELTA
0,1 AV L BMT,2016-12-31,7,566.0
1,1 AV L BMT,2016-12-31,11,1880.0
2,1 AV L BMT,2016-12-31,15,3329.0
3,1 AV L BMT,2016-12-31,19,5305.0
4,1 AV L BMT,2016-12-31,23,5172.0


In [7]:
mta_df = mta_df.rename(index=str, columns={'EXITS                                                               ': "EXITS"})

In [8]:
mta_df['DELTA'] = mta_df['EXITS'].diff().abs()

In [9]:
mta_df = mta_df[mta_df.time_of_day != 3]

In [10]:
mta_df = mta_df[mta_df.DELTA < 8600]

In [11]:
mta_df['day_of_the_month'] = pd.to_datetime(mta_df.DATE).dt.day

In [12]:
agg_mta = mta_df.groupby(['zone', 'DATE', 'time_of_day'])['DELTA'].sum().reset_index()

In [13]:
agg_mta.head()

,zone,DATE,time_of_day,DELTA
0,1 AV L BMT,2016-12-31,7,566.0
1,1 AV L BMT,2016-12-31,11,1880.0
2,1 AV L BMT,2016-12-31,15,3329.0
3,1 AV L BMT,2016-12-31,19,5305.0
4,1 AV L BMT,2016-12-31,23,5172.0


In [13]:
bike_df[bike_df.zone == 'False']

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,month,year,day_of_week,time_of_day,zone,DATE


In [40]:
bike_df = bike_df[bike_df.zone != 'False']

In [61]:
f = open('storage/test_bike.pckl', 'wb')
pickle.dump(bike_df, f)
f.close()

In [14]:
f = open('storage/test_mta.pckl', 'wb')
pickle.dump(agg_mta, f)
f.close()

In [4]:
f = open('storage/test_bike.pckl', 'rb')
bike_df = pickle.load(f)
f.close()

In [5]:
f = open('storage/test_mta.pckl', 'rb')
mta_df = pickle.load(f)
f.close()

In [31]:
test = pd.merge(bike_df, mta_df, on=['zone','DATE','time_of_day'])

In [33]:
# test.groupby(['zone','DATE','time_of_day'])['Bike ID'].count()

In [35]:
# bike_df.groupby(['zone', 'DATE','time_of_day'])['Bike ID'].count().reset_index()

In [4]:
mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,month,year,day_of_week,time_of_day,lat,long,zone
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 03:00:00,REGULAR,5991546,2028378,12,2016,Saturday,3,40.641568,-74.017678,59 ST NQR456W BMT
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 07:00:00,REGULAR,5991565,2028389,12,2016,Saturday,7,40.641568,-74.017678,59 ST NQR456W BMT
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 11:00:00,REGULAR,5991644,2028441,12,2016,Saturday,11,40.641568,-74.017678,59 ST NQR456W BMT
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 15:00:00,REGULAR,5991971,2028502,12,2016,Saturday,15,40.641568,-74.017678,59 ST NQR456W BMT
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2016-12-31,1900-01-01 19:00:00,REGULAR,5992418,2028543,12,2016,Saturday,19,40.641568,-74.017678,59 ST NQR456W BMT


In [16]:
test = pd.merge(agg_mta, bike_df, on=['zone', 'DATE', 'time_of_day'])

In [15]:
agg_mta.groupby(['zone', 'DATE', 'time_of_day']).count().reset_index().head()

,zone,DATE,time_of_day,DELTA
0,1 AV L BMT,2016-12-31,7,1
1,1 AV L BMT,2016-12-31,11,1
2,1 AV L BMT,2016-12-31,15,1
3,1 AV L BMT,2016-12-31,19,1
4,1 AV L BMT,2016-12-31,23,1


## Testing

In [17]:
test.head()

,zone,DATE,time_of_day,DELTA,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,month,year,day_of_week
0,1 AV L BMT,2017-01-01,11,1063.0,595,2017-01-01 07:39:58,2017-01-01 07:49:54,3376,E 65 St & 2 Ave,40.764719,-73.962221,518,E 39 St & 2 Ave,40.747804,-73.973442,25253,Subscriber,1983.0,1,1,2017,Sunday
1,1 AV L BMT,2017-01-01,11,1063.0,148,2017-01-01 07:42:24,2017-01-01 07:44:52,3376,E 65 St & 2 Ave,40.764719,-73.962221,3142,1 Ave & E 62 St,40.761227,-73.960940,19101,Subscriber,1987.0,1,1,2017,Sunday
2,1 AV L BMT,2017-01-01,11,1063.0,294,2017-01-01 07:42:46,2017-01-01 07:47:40,3141,1 Ave & E 68 St,40.765005,-73.958185,3155,Lexington Ave & E 63 St,40.764400,-73.966490,14881,Subscriber,1986.0,1,1,2017,Sunday
3,1 AV L BMT,2017-01-01,11,1063.0,712,2017-01-01 07:46:37,2017-01-01 07:58:29,3141,1 Ave & E 68 St,40.765005,-73.958185,527,E 33 St & 2 Ave,40.744023,-73.976056,23442,Subscriber,1988.0,1,1,2017,Sunday
4,1 AV L BMT,2017-01-01,11,1063.0,2170,2017-01-01 08:59:28,2017-01-01 09:35:39,3141,1 Ave & E 68 St,40.765005,-73.958185,3141,1 Ave & E 68 St,40.765005,-73.958185,15828,Subscriber,1978.0,1,1,2017,Sunday


In [18]:
test.shape

(174469, 22)

In [42]:
ini = test.groupby(['zone', 'DATE', 'time_of_day'])

In [43]:
df = ini['Bike ID'].count().reset_index()

In [44]:
ini['DELTA'].sum().reset_index().head()

,zone,DATE,time_of_day,DELTA
0,1 AV L BMT,2017-01-01,11,6378.0
1,1 AV L BMT,2017-01-01,15,45720.0
2,1 AV L BMT,2017-01-01,19,57075.0
3,1 AV L BMT,2017-01-01,23,45934.0
4,1 AV L BMT,2017-01-02,11,17667.0


In [46]:
df.head()

,zone,DATE,time_of_day,Bike ID
0,1 AV L BMT,2017-01-01,11,6
1,1 AV L BMT,2017-01-01,15,20
2,1 AV L BMT,2017-01-01,19,15
3,1 AV L BMT,2017-01-01,23,17
4,1 AV L BMT,2017-01-02,11,9


In [47]:
df = pd.merge(df, agg_mta, on=['zone', 'DATE', 'time_of_day'])

In [69]:
df['DATE'] = test.DATE.to_string

In [27]:
streets = df.zone.unique()

In [48]:
def dummify(df):
    dummies = pd.get_dummies(df['zone'])
    df[dummies.columns] = dummies
    return df

df = dummify(df)

In [37]:
col = ['time_of_day', '1 AV L BMT', '103 ST 1 IRT',
       '103 ST 6 IRT', '14 ST 123FLM IRT', '14 ST-UNION SQ 456LNQRW IRT',
       '15 ST-PROSPECT FG IND', '2 AV F IND', '21 ST-QNSBRIDGE F IND',
       '23 ST 1 IRT', '23 ST 6 IRT', '23 ST CE IND', '23 ST NRW BMT',
       '28 ST 6 IRT', '3 AV L BMT', '33 ST 6 IRT',
       '34 ST-HERALD SQ BDFMNQRW BMT', '34 ST-HUDSON YD 7 IRT',
       '34 ST-PENN STA 123 IRT', '4 AV-9 ST DFGMNR IND',
       '42 ST-BRYANT PK BDFM7 IND', '47-50 STS ROCK BDFM IND',
       '49 ST NQRW BMT', '5 AV/53 ST EM IND', '5 AV/59 ST NQRW BMT',
       '5 AVE 7BDFM IRT', '50 ST 1 IRT', '50 ST CE IND', '51 ST 6 IRT',
       '57 ST-7 AV NQRW BMT', '59 ST COLUMBUS 1ABCD IRT', '6 AV FLM123 BMT',
       '7 AV FG IND', '72 ST 123 IRT', '77 ST 6 IRT', '8 AV ACEL BMT',
       '8 ST-NYU NRW BMT', '86 ST 1 IRT', '86 ST BC IND', '96 ST 123 IRT',
       '96 ST 6 IRT', '96 ST BC IND', '9TH STREET 1 PTH',
       'ATL AV-BARCLAY 2345BDNQR IRT', 'B\'WAY-LAFAYETTE BDFQ6 IND',
       'BEDFORD AV L BMT', 'BEDFORD-NOSTRAN G IND', 'BERGEN ST 23 IRT',
       'BLEECKER ST 6DF IRT', 'BOROUGH HALL 2345R IRT',
       'BOROUGH HALL R2345 BMT', 'BOWERY JZ BMT', 'BOWLING GREEN 45 IRT',
       'CANAL ST ACE IND', 'CARROLL ST FG IND', 'CATHEDRAL PKWY 1 IRT',
       'CHAMBERS ST 123 IRT', 'CHRISTOPHER ST 1 IRT', 'CHRISTOPHER ST 1 PTH',
       'CITY / BUS 1 PTH', 'CLARK ST 23 IRT', 'CLASSON AV G IND',
       'CLINTON-WASH AV C IND', 'CORTLANDT ST RNW BMT', 'COURT SQ 7 IRT',
       'DEKALB AV BDNQR BMT', 'EAST BROADWAY F IND', 'FLUSHING AV G IND',
       'FRANKLIN AV 2345S IRT', 'FRANKLIN ST 1 IRT', 'GRAHAM AV L BMT',
       'GRAND ARMY PLAZ 23 IRT', 'GRAND ST BD IND', 'GRAND ST L BMT',
       'GRD CNTRL-42 ST 4567S IRT', 'GREENPOINT AV G IND', 'HIGH ST AC IND',
       'HOYT-SCHER ACG IND', 'KINGSTON-THROOP C IND',
       'LEXINGTON AV/53 EM6 IND', 'LEXINGTON AV/63 F IND', 'MONTROSE AV L BMT',
       'MYRTLE AV JMZ BMT', 'MYRTLE-WILLOUGH G IND', 'NEVINS ST 2345 IRT',
       'PRESIDENT ST 25 IRT', 'PRINCE ST NRW BMT', 'QUEENSBORO PLZ 7NQW IRT',
       'SMITH-9 ST FG IND', 'SPRING ST 6 IRT', 'SPRING ST CE IND',
       'UNION ST R BMT', 'VERNON-JACKSON 7 IRT', 'WALL ST 23 IRT']

In [49]:
df.head()

,zone,DATE,time_of_day,Bike ID,DELTA,1 AV L BMT,103 ST 1 IRT,103 ST 6 IRT,14 ST 123FLM IRT,14 ST-UNION SQ 456LNQRW IRT,15 ST-PROSPECT FG IND,2 AV F IND,21 ST-QNSBRIDGE F IND,23 ST 1 IRT,23 ST 6 IRT,23 ST CE IND,23 ST NRW BMT,28 ST 6 IRT,3 AV L BMT,33 ST 6 IRT,34 ST-HERALD SQ BDFMNQRW BMT,34 ST-HUDSON YD 7 IRT,34 ST-PENN STA 123 IRT,4 AV-9 ST DFGMNR IND,42 ST-BRYANT PK BDFM7 IND,47-50 STS ROCK BDFM IND,49 ST NQRW BMT,5 AV/53 ST EM IND,5 AV/59 ST NQRW BMT,5 AVE 7BDFM IRT,50 ST 1 IRT,50 ST CE IND,51 ST 6 IRT,57 ST-7 AV NQRW BMT,59 ST COLUMBUS 1ABCD IRT,6 AV FLM123 BMT,7 AV FG IND,72 ST 123 IRT,77 ST 6 IRT,8 AV ACEL BMT,8 ST-NYU NRW BMT,86 ST 1 IRT,86 ST BC IND,96 ST 123 IRT,96 ST 6 IRT,96 ST BC IND,9TH STREET 1 PTH,ATL AV-BARCLAY 2345BDNQR IRT,B'WAY-LAFAYETTE BDFQ6 IND,BEDFORD AV L BMT,BEDFORD-NOSTRAN G IND,BERGEN ST 23 IRT,BLEECKER ST 6DF IRT,BOROUGH HALL 2345R IRT,BOROUGH HALL R2345 BMT,BOWERY JZ BMT,BOWLING GREEN 45 IRT,CANAL ST ACE IND,CARROLL ST FG IND,CATHEDRAL PKWY 1 IRT,CHAMBERS ST 123 IRT,CHRISTOPHER ST 1 IRT,CHRISTOPHER ST 1 PTH,CITY / BUS 1 PTH,CLARK ST 23 IRT,CLASSON AV G IND,CLINTON-WASH AV C IND,CORTLANDT ST RNW BMT,COURT SQ 7 IRT,DEKALB AV BDNQR BMT,EAST BROADWAY F IND,FLUSHING AV G IND,FRANKLIN AV 2345S IRT,FRANKLIN ST 1 IRT,GRAHAM AV L BMT,GRAND ARMY PLAZ 23 IRT,GRAND ST BD IND,GRAND ST L BMT,GRD CNTRL-42 ST 4567S IRT,GREENPOINT AV G IND,HIGH ST AC IND,HOYT-SCHER ACG IND,KINGSTON-THROOP C IND,LEXINGTON AV/53 EM6 IND,LEXINGTON AV/63 F IND,MONTROSE AV L BMT,MYRTLE AV JMZ BMT,MYRTLE-WILLOUGH G IND,NEVINS ST 2345 IRT,PRESIDENT ST 25 IRT,PRINCE ST NRW BMT,QUEENSBORO PLZ 7NQW IRT,SMITH-9 ST FG IND,SPRING ST 6 IRT,SPRING ST CE IND,UNION ST R BMT,VERNON-JACKSON 7 IRT,WALL ST 23 IRT
0,1 AV L BMT,2017-01-01,11,6,1063.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1 AV L BMT,2017-01-01,15,20,2286.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1 AV L BMT,2017-01-01,19,15,3805.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1 AV L BMT,2017-01-01,23,17,2702.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1 AV L BMT,2017-01-02,11,9,1963.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
def randomforestmodel(df):
    print("Running Model...")
    df = df.sort_values('DATE')
    
    df['DATE'] = df.DATE.to_string
    
    rows = df.shape[0]

    train = df.head(int(rows*0.8))
    test = df.tail(int(rows*0.2))

    rfrmodel = RandomForestRegressor(n_estimators=20, n_jobs=-1)
    reg = rfrmodel.fit(train[col], train['Bike ID'])

    training_accuracy = reg.score(train[col], train['Bike ID'])
    test_accuracy = reg.score(test[col], test['Bike ID'])

    log_error = mean_squared_log_error(test['Bike ID'], rfrmodel.predict(test[col]))
#     print(rfrmodel.predict(test[col]))
#     print(test['Bike ID'])
    print("############# based on standard predict ################")
    print("R^2 on training data: %0.4f" % (training_accuracy))
    print("R^2 on test data:     %0.4f" % (test_accuracy))
    print("log on test data:     %0.4f" % (log_error))

In [52]:
randomforestmodel(df)

Running Model...
############# based on standard predict ################
R^2 on training data: 0.5845
R^2 on test data:     0.6335
log on test data:     0.3785


In [40]:
df.head()

,zone,DATE,time_of_day,Bike ID,DELTA,1 AV L BMT
0,1 AV L BMT,2017-01-01,11,6,1063.0,1
1,1 AV L BMT,2017-01-01,15,20,2286.0,1
2,1 AV L BMT,2017-01-01,19,15,3805.0,1
3,1 AV L BMT,2017-01-01,23,17,2702.0,1
4,1 AV L BMT,2017-01-02,11,9,1963.0,1


In [51]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
predict the number of users that use the citibike after getting of at a subway station. 

Running Model...
############# based on standard predict ################
R^2 on training data: 0.5841
R^2 on test data:     0.6292
log on test data:     0.3791


In [ ]:
col = [ 'time_of_day', 'DELTA', '1 AV L BMT',
     '103 ST 6 IRT', '14 ST 123FLM IRT',
       '14 ST-UNION SQ 456LNQRW IRT', '15 ST-PROSPECT FG IND', '2 AV F IND',
       '21 ST-QNSBRIDGE F IND', '23 ST 1 IRT', '23 ST 6 IRT', '23 ST CE IND',
       '23 ST NRW BMT', '28 ST 6 IRT', '3 AV L BMT', '33 ST 6 IRT',
       '34 ST-HERALD SQ BDFMNQRW BMT', '34 ST-HUDSON YD 7 IRT',
       '34 ST-PENN STA 123 IRT', '4 AV-9 ST DFGMNR IND',
       '42 ST-BRYANT PK BDFM7 IND', '47-50 STS ROCK BDFM IND',
       '49 ST NQRW BMT', '5 AV/53 ST EM IND', '5 AV/59 ST NQRW BMT',
       '5 AVE 7BDFM IRT', '50 ST 1 IRT', '50 ST CE IND', '51 ST 6 IRT',
       '57 ST-7 AV NQRW BMT', '59 ST COLUMBUS 1ABCD IRT', '6 AV FLM123 BMT',
       '7 AV FG IND', '72 ST 123 IRT', '77 ST 6 IRT', '8 AV ACEL BMT',
       '8 ST-NYU NRW BMT', '86 ST 1 IRT', '86 ST BC IND', '96 ST 123 IRT',
       '96 ST 6 IRT', '96 ST BC IND', '9TH STREET 1 PTH',
       'ATL AV-BARCLAY 2345BDNQR IRT', 'B\'WAY-LAFAYETTE BDFQ6 IND',
       'BEDFORD AV L BMT', 'BEDFORD-NOSTRAN G IND', 'BERGEN ST 23 IRT',
       'BLEECKER ST 6DF IRT', 'BOROUGH HALL 2345R IRT',
       'BOROUGH HALL R2345 BMT', 'BOWERY JZ BMT', 'BOWLING GREEN 45 IRT',
       'CANAL ST ACE IND', 'CARROLL ST FG IND', 'CATHEDRAL PKWY 1 IRT',
       'CHAMBERS ST 123 IRT', 'CHRISTOPHER ST 1 IRT', 'CHRISTOPHER ST 1 PTH',
       'CITY / BUS 1 PTH', 'CLARK ST 23 IRT', 'CLASSON AV G IND',
       'CLINTON-WASH AV C IND', 'CORTLANDT ST RNW BMT', 'COURT SQ 7 IRT',
       'DEKALB AV BDNQR BMT', 'EAST BROADWAY F IND', 'FLUSHING AV G IND',
       'FRANKLIN AV 2345S IRT', 'FRANKLIN ST 1 IRT', 'GRAHAM AV L BMT',
       'GRAND ARMY PLAZ 23 IRT', 'GRAND ST BD IND', 'GRAND ST L BMT',
       'GRD CNTRL-42 ST 4567S IRT', 'GREENPOINT AV G IND', 'HIGH ST AC IND',
       'HOYT-SCHER ACG IND', 'KINGSTON-THROOP C IND',
       'LEXINGTON AV/53 EM6 IND', 'LEXINGTON AV/63 F IND', 'MONTROSE AV L BMT',
       'MYRTLE AV JMZ BMT', 'MYRTLE-WILLOUGH G IND', 'NEVINS ST 2345 IRT',
       'PRESIDENT ST 25 IRT', 'PRINCE ST NRW BMT', 'QUEENSBORO PLZ 7NQW IRT',
       'SMITH-9 ST FG IND', 'SPRING ST 6 IRT', 'SPRING ST CE IND',
       'UNION ST R BMT', 'VERNON-JACKSON 7 IRT', 'WALL ST 23 IRT']

